# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [4]:
# Importing libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [5]:
# your code here

sentiment = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
sentiment.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
sentiment_sample = sentiment.sample(10000)
sentiment_sample = sentiment_sample.rename(columns={0 : 'target', 1 : 'id', 2 : 'date', 3: 'flag', 4 : 'user', 
                        5: 'text'})

In [7]:
sentiment_sample.head()

,target,id,date,flag,user,text
761363,0,2297203781,Tue Jun 23 10:09:12 PDT 2009,NO_QUERY,chebs09,knocking yourself unconscious isnt all that! t...
1498260,4,2070542788,Sun Jun 07 17:52:57 PDT 2009,NO_QUERY,alyssafrey,@yaymarissa less than 1 page
425966,0,2063435750,Sun Jun 07 02:38:59 PDT 2009,NO_QUERY,ohmygarters,@LAUR4T GOOD LUCK! I wanted to be there
1218681,4,1989861385,Mon Jun 01 03:04:03 PDT 2009,NO_QUERY,PaulHeylin,@MontinaCooper hey are you touring with beyon...
1345910,4,2044114680,Fri Jun 05 08:53:29 PDT 2009,NO_QUERY,MauraAnderson,@jyscale Will do - basically dropped off my ca...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [8]:
# Importing the libraries:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re

In [9]:
# Using the functions:
def clean_up(s):
    w = re.sub(r'http\S+', ' ', s).replace("'", " ").lower()
    w = re.sub(r"[^a-zA-Z]", " ", w)
    return w

def tokenize(s):
    w = nltk.word_tokenize(s)
    return w   

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    a = [lemmatizer.lemmatize(ps.stem(w)) for w in l]
    return a

def remove_stopwords(s):
    stop_words = stopwords.words('english')
    w = [x for x in s if x not in stop_words]
    return w

In [10]:
# your code here
sentiment_sample['text_processed'] = sentiment_sample.apply(lambda x : remove_stopwords(
    stem_and_lemmatize(tokenize(clean_up(x['text'])))), axis = 1)

In [11]:
sentiment_sample.head(10)

,target,id,date,flag,user,text,text_processed
761363,0,2297203781,Tue Jun 23 10:09:12 PDT 2009,NO_QUERY,chebs09,knocking yourself unconscious isnt all that! t...,"[knock, unconsci, isnt, star, pretti, tho]"
1498260,4,2070542788,Sun Jun 07 17:52:57 PDT 2009,NO_QUERY,alyssafrey,@yaymarissa less than 1 page,"[yaymarissa, le, page]"
425966,0,2063435750,Sun Jun 07 02:38:59 PDT 2009,NO_QUERY,ohmygarters,@LAUR4T GOOD LUCK! I wanted to be there,"[laur, good, luck, want]"
1218681,4,1989861385,Mon Jun 01 03:04:03 PDT 2009,NO_QUERY,PaulHeylin,@MontinaCooper hey are you touring with beyon...,"[montinacoop, hey, tour, beyonc]"
1345910,4,2044114680,Fri Jun 05 08:53:29 PDT 2009,NO_QUERY,MauraAnderson,@jyscale Will do - basically dropped off my ca...,"[jyscal, basic, drop, car, wait, phone, call, ..."
1237528,4,1993055893,Mon Jun 01 09:45:35 PDT 2009,NO_QUERY,TheNudel,http://twitpic.com/6eq87 - My brother and his ...,"[brother, hi, girlfreind]"
65049,0,1690791809,Sun May 03 16:23:18 PDT 2009,NO_QUERY,Lydiapie,hate the fact that i'm miles away from my hunn...,"[hate, fact, mile, away, hunni]"
1254940,4,1997195720,Mon Jun 01 16:25:25 PDT 2009,NO_QUERY,KristenCampisi,@Patricia_Knight ;) Yeah....yeah...I gotcha.....,"[patricia, knight, yeah, yeah, gotcha, ok, wan..."
978588,4,1833849762,Mon May 18 00:40:43 PDT 2009,NO_QUERY,marissazaguirre,46 followers. Crazyyy. Well for me at least. H...,"[follow, crazyyi, well, least, haha, let, see,..."
1540360,4,2180424827,Mon Jun 15 10:27:10 PDT 2009,NO_QUERY,Nimhra,Insurance kicked in and im getting a $42 check,"[insur, kick, im, get, check]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [13]:
# your code here
from nltk.probability import FreqDist
words = [w for row in sentiment_sample['text_processed'] for w in row]
fdist = FreqDist(words)
fdist

FreqDist({'go': 944, 'day': 680, 'get': 675, 'wa': 649, 'thi': 578, 'good': 569, 'work': 525, 'like': 524, 'love': 482, 'quot': 442, ...})

In [14]:
top_words = [w[0] for w in fdist.most_common(5000)]
top_words

['go',
 'day',
 'get',
 'wa',
 'thi',
 'good',
 'work',
 'like',
 'love',
 'quot',
 'got',
 'today',
 'u',
 'time',
 'miss',
 'back',
 'thank',
 'one',
 'lol',
 'want',
 'know',
 'see',
 'feel',
 'im',
 'night',
 'realli',
 'think',
 'watch',
 'need',
 'still',
 'hope',
 'make',
 'na',
 'last',
 'oh',
 'well',
 'amp',
 'look',
 'home',
 'ha',
 'new',
 'come',
 'great',
 'much',
 'sleep',
 'wish',
 'morn',
 'fun',
 'twitter',
 'wait',
 'follow',
 'sad',
 'haha',
 'happi',
 'tomorrow',
 'veri',
 'onli',
 'bad',
 'tri',
 'week',
 'say',
 'sorri',
 'thing',
 'friend',
 'whi',
 'right',
 'tonight',
 'hate',
 'though',
 'gon',
 'would',
 'hi',
 'way',
 'hour',
 'better',
 'take',
 'show',
 'yeah',
 'nice',
 'could',
 'tweet',
 'start',
 'school',
 'even',
 'peopl',
 'final',
 'bed',
 'next',
 'weekend',
 'let',
 'awesom',
 'soon',
 'hey',
 'x',
 'ye',
 'cant',
 'best',
 'movi',
 'guy',
 'someth',
 'play',
 'wan',
 'sick',
 'ok',
 'lt',
 'keep',
 'sure',
 'anoth',
 'rain',
 'use',
 'littl',
 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [15]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in fdist:
        features[w] = (w in words)
    return features

In [16]:
docs = list(zip(sentiment_sample['text_processed'], np.where(sentiment_sample['target'] == 4, True, False)))

featuresets = [(find_features(rev), category) for (rev, category) in docs]
featuresets[0]

({'knock': True,
  'unconsci': True,
  'isnt': True,
  'star': True,
  'pretti': True,
  'tho': True,
  'yaymarissa': False,
  'le': False,
  'page': False,
  'laur': False,
  'good': False,
  'luck': False,
  'want': False,
  'montinacoop': False,
  'hey': False,
  'tour': False,
  'beyonc': False,
  'jyscal': False,
  'basic': False,
  'drop': False,
  'car': False,
  'wait': False,
  'phone': False,
  'call': False,
  'say': False,
  'done': False,
  'brother': False,
  'hi': False,
  'girlfreind': False,
  'hate': False,
  'fact': False,
  'mile': False,
  'away': False,
  'hunni': False,
  'patricia': False,
  'knight': False,
  'yeah': False,
  'gotcha': False,
  'ok': False,
  'spillag': False,
  'could': False,
  'live': False,
  'cereal': False,
  'love': False,
  'told': False,
  'ya': False,
  'im': False,
  'kid': False,
  'follow': False,
  'crazyyi': False,
  'well': False,
  'least': False,
  'haha': False,
  'let': False,
  'see': False,
  'get': False,
  'end': False,


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [17]:
# your code here
n = int(len(featuresets) / 2)
training_set = featuresets[:n]
testing_set = featuresets[n:]
classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features(20)

Most Informative Features
                  welcom = True             True : False  =     16.4 : 1.0
                 headach = True            False : True   =     14.9 : 1.0
                     sad = True            False : True   =     12.5 : 1.0
                      aw = True            False : True   =      8.2 : 1.0
                    sick = True            False : True   =      7.7 : 1.0
                    lone = True            False : True   =      7.6 : 1.0
                   freak = True            False : True   =      7.6 : 1.0
                  throat = True            False : True   =      7.6 : 1.0
              disappoint = True            False : True   =      7.6 : 1.0
                     die = True            False : True   =      7.4 : 1.0
                question = True             True : False  =      7.0 : 1.0
                     add = True             True : False  =      7.0 : 1.0
                  cancel = True            False : True   =      7.0 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here
nltk.classify.accuracy(classifier, testing_set)

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here